### 1) Загрузка таблиц по ссылкам и объединение ###

In [60]:
import pandas as pd
import numpy as np
from IPython.display import display

url_1 = "https://pstu.ru/files/file/Abitur/2025%20final/%D0%9C%D0%A2%D0%A4_%D0%91_%D0%9D_000007045.html"

# Загружаем таблицу
raw_df = pd.read_html(url_1, header=0, skiprows=12)[0].copy()
display(raw_df)


,№,Уникальный код,Сумма баллов,АиНМА / Математика,ПФ / Информатика и ИКТ / Физика / Химия,Русский язык,Сумма баллов за ИД,Учебная группа,Представление приказа,Идентификационный номер заказчика целевого обучения (для целевого приема),Номер предложения (для целевого приема),Unnamed: 11,Unnamed: 12
0,1,4430320,255,78,88,89,0,КТОМ-25-2б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
1,2,4982142,247,82,100,65,0,КТОМ-25-2б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
2,3,3870807,238,72,75,86,5,КТОМ-25-2б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
3,4,4657367,237,80,71,86,0,КТОМ-25-2б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
4,5,4243572,235,80,74,81,0,КТОМ-25-3б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
5,6,3943378,233,76,79,78,0,КТОМ-25-3б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
6,7,4408104,231,80,76,75,0,КТОМ-25-3б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
7,8,3984986,228,72,67,89,0,КТОМ-25-3б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
8,9,4046083,226,80,82,64,0,КТОМ-25-3б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN
9,10,4008326,224,80,71,73,0,КТОМ-25-3б,№ 07-08/7-С от 07.08.2025,NaN,NaN,NaN,NaN


### 2) Приведение к "рабочему" виду ###

In [61]:
df = raw_df.copy()

# Заменяем сложные названия предметов на простые
df.columns = [str(c).strip().replace("\n", " ") for c in df.columns]
rename_map = {}
for c in df.columns:
    if ('АиНМА' in c) or ('Математика' in c):
        rename_map[c] = 'Математика'
    elif any(k in c for k in ['ПФ', 'География', 'Информатика и ИКТ', 'Физика', 'Химия']):
        rename_map[c] = 'Профильный предмет'
    elif 'Русский' in c:
        rename_map[c] = 'Русский язык'
df = df.rename(columns=rename_map)

# Удаляем полностью пустые столбцы
df = df.dropna(axis=1, how='all')

# Нормализуем заголовки
df.columns = [str(c).strip().replace('\n', ' ') for c in df.columns]


# Удаляем служебные столбцы, если присутствуют
cols_to_drop = [
    '№',
    'Уникальный код',
    'Представление приказа',
    'Учебная группа',
    'Идентификационный номер заказчика целевого обучения (для целевого приема)',
    'Номер предложения (для целевого приема)'
]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# Переустанавливаем индексацию
df = df.reset_index(drop=True)
df.index = df.index + 1
df.index.name = '№'

display(df.head(15))


,Сумма баллов,Математика,Профильный предмет,Русский язык,Сумма баллов за ИД
№,,,,,
1,255,78,88,89,0
2,247,82,100,65,0
3,238,72,75,86,5
4,237,80,71,86,0
5,235,80,74,81,0
6,233,76,79,78,0
7,231,80,76,75,0
8,228,72,67,89,0
9,226,80,82,64,0


### 3) Проверка типов и приведение количественных к числовому типу ###

In [70]:
print("До преобразования:")
print(df.dtypes)

numeric_candidates = [
    c for c in df.columns
    if any(k in c for k in ['Сумма баллов', 'Математика', 'Профильный предмет', 'Русский'])
]

for c in numeric_candidates:
    df[c] = pd.to_numeric(
        df[c].astype(str).str.replace(',', '.', regex=False),
        errors='coerce'
    )

print("\nПосле преобразования:")
print(df[numeric_candidates].dtypes)

display(df)


До преобразования:
Сумма баллов          int64
Математика            int64
Профильный предмет    int64
Русский язык          int64
Сумма баллов за ИД    int64
dtype: object

После преобразования:
Сумма баллов          int64
Математика            int64
Профильный предмет    int64
Русский язык          int64
Сумма баллов за ИД    int64
dtype: object


,Сумма баллов,Математика,Профильный предмет,Русский язык,Сумма баллов за ИД
№,,,,,
1,255,78,88,89,0
2,247,82,100,65,0
3,238,72,75,86,5
4,237,80,71,86,0
5,235,80,74,81,0
6,233,76,79,78,0
7,231,80,76,75,0
8,228,72,67,89,0
9,226,80,82,64,0


### 4) Средний, минимальный и максимальный суммарные баллы ###

In [69]:
if 'Сумма баллов' not in df.columns:
    raise ValueError("Колонка 'Сумма баллов' не найдена.")

mean_total = df['Сумма баллов'].mean()
min_total = df['Сумма баллов'].min()
max_total = df['Сумма баллов'].max()

print(f"Средняя сумма баллов: {mean_total:.2f}")
print("Минимальная сумма баллов:", min_total)
print("Максимальная сумма баллов:", max_total)
display(df)


Средняя сумма баллов: 203.58
Минимальная сумма баллов: 181
Максимальная сумма баллов: 255


,Сумма баллов,Математика,Профильный предмет,Русский язык,Сумма баллов за ИД
№,,,,,
1,255,78,88,89,0
2,247,82,100,65,0
3,238,72,75,86,5
4,237,80,71,86,0
5,235,80,74,81,0
6,233,76,79,78,0
7,231,80,76,75,0
8,228,72,67,89,0
9,226,80,82,64,0


### 5) Средние по трём предметам и предмет с наибольшим средним ###

In [64]:
required_cols = ['Предмет №1', 'Предмет №2', 'Русский язык']
missing = [c for c in required_cols if c not in df.columns]

means_df = pd.DataFrame({
    'Предмет №1': [df['Математика'].mean()],
    'Предмет №2': [df['Профильный предмет'].mean()],
    'Русский язык': [df['Русский язык'].mean()],
}, index=['Среднее'])

display(means_df)

top_subject = means_df.idxmax(axis=1).values[0]
print("Предмет с наибольшим средним баллом:", top_subject)


,Предмет №1,Предмет №2,Русский язык
Среднее,68.422222,67.733333,67.2


Предмет с наибольшим средним баллом: Предмет №1


### 6) Количество абитуриентов Тип_1 ###

In [68]:
candidates = {c: c for c in df.columns if c in ['Русский язык', 'Математика', 'Профильный предмет']}
rus_col = candidates.get('Русский язык')
subj1_col = candidates.get('Математика')
subj2_col = candidates.get('Профильный предмет')

rus_mean = df[rus_col].mean() if rus_col else np.nan
subj1_mean = df[subj1_col].mean() if subj1_col else np.nan
subj2_mean = df[subj2_col].mean() if subj2_col else np.nan

if all([rus_col, subj1_col, subj2_col]):
    type_1 = int(((df[rus_col] < rus_mean) & (df[subj1_col] > subj1_mean) & (df[subj2_col] > subj2_mean)).sum())
else:
    type_1 = 0

print("Кол-во студентов Тип_1:", type_1)


Кол-во студентов Тип_1: 9


### 7) Количество абитуриентов Тип_2 ###

In [66]:
if all([rus_col, subj1_col, subj2_col]):
    type_2 = int(((df[rus_col] > rus_mean) & (df[subj1_col] < subj1_mean) & (df[subj2_col] < subj2_mean)).sum())
else:
    type_2 = 0

print("Кол-во студентов Тип_2:", type_2)


Кол-во студентов Тип_2: 4


### 8) Итоговый DataFrame с метаданными ###

In [67]:
summary_df = pd.DataFrame([
    {
        "Подразделение": "Механико-технологический факультет",
        "Уровень подготовки": "Бакалавриат",
        "Направление подготовки/специальность": "Конструкторско-технологическое обеспечение машиностроительных производств",
        "Год": 2025,
        "Количество мест": int(len(df)),
        "Предметы ЕГЭ": "Математика, Профильный предмет, Русский язык",
        "Средняя сумма баллов": float(mean_total) if pd.notna(mean_total) else None,
        "Min сумма баллов": float(min_total) if pd.notna(min_total) else None,
        "Max сумма баллов": float(max_total) if pd.notna(max_total) else None,
        "Предмет с высшим средним": top_subject,
        "Кол-во студентов тип_1": int(type_1),
        "Кол-во студентов тип_2": int(type_2),
    }
])

display(summary_df)


,Подразделение,Уровень подготовки,Направление подготовки/специальность,Год,Количество мест,Предметы ЕГЭ,Средняя сумма баллов,Min сумма баллов,Max сумма баллов,Предмет с высшим средним,Кол-во студентов тип_1,Кол-во студентов тип_2
0,Механико-технологический факультет,Бакалавриат,Конструкторско-технологическое обеспечение маш...,2025,45,"Математика, Профильный предмет, Русский язык",203.577778,181.0,255.0,Предмет №1,9,4
